In [1]:
%reload_ext autoreload
%reload_ext autoreload

In [2]:
import time
import os
import sys
import copy
import time
import datetime
import random
import math
import warnings
from functools import partial
# warnings.filterwarnings('ignore')

In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [4]:
import numpy as np
import scipy
import seaborn as sns

from tqdm.notebook import tqdm, trange

In [5]:
sys.path.append("../")

In [6]:
from chaosmining.data_utils import read_formulas, create_simulation_data
from chaosmining.simulation.models import MLPRegressor
from chaosmining.simulation.functions import abs_argmax_topk
from chaosmining.utils import radar_factory

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [8]:
from captum.attr import IntegratedGradients, Saliency, DeepLift, FeatureAblation

In [9]:
import matplotlib
# mpl.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt 
from matplotlib.colors import ListedColormap

matplotlib.rcParams['lines.linewidth'] = 1
matplotlib.rcParams['lines.markersize'] = 5
plt.rcParams['figure.figsize'] = [4, 4]

# Generate Functions and Data

In [10]:
formulas_path = '../data/symbolic_simulation/formula.csv'

In [11]:
formulas = read_formulas(formulas_path)

In [12]:
formulas

[[1, 'a'],
 [1, 'a**2'],
 [1, '2/(a**2+1)-1'],
 [1, 'sin(a)'],
 [1, 'E**a-1.5'],
 [1, '2*log(a**2+1)-1'],
 [2, '0.25*a**3+0.75*b**2'],
 [3, '0.5*a**3+0.75*b**2+a*c'],
 [4, '0.5*E**a*sin(b)-0.25*cos(d)**5/(c**2+1)'],
 [5, '0.5*(a-b)**2+0.2*(c+d*e)**3-0.5'],
 [6, '0.5*cos(a)*tan(b)-log((c-d)**2+1)/((e+f+1)**2+1)'],
 [7, '0.5*(b-c)**2/(a**2+1)+tan(d)*log(e**2+1)+0.5*cos(f)*sin(g)'],
 [8,
  'a+(1/2)*b**2+(1/3)*c**3+(1/4)*d**4+(1/5)*e**5+(1/6)*f**6+(1/7)*f**7+(1/8)*g**8'],
 [9,
  '0.5*(a-1)/(b**2+1)-0.5*c**3/(d**2+1)+0.5*e**5/(f**2+1)-0.5*g**7/(h**2+1)+0.5*tan(i)+0.5'],
 [10,
  '0.5*sin(a)-0.5*b**3-log(c**2+1)+0.5*((d+e)**2+1)**(1/2)-0.5*cos(f)*g+h*i**2/((1-j)**2+1)-0.5']]

In [13]:
formula_id = 14
seed = 42
test_size = 0.2
n_steps = 20

In [14]:
num_features, function = formulas[formula_id]

In [16]:
num_noises = 150
num_data = 10000
X_var = 0.33
y_var = 0.01
X, y_true, y_noise, intercepts, derivatives, integrations = create_simulation_data(function, num_features, num_noises, num_data, X_var, y_var, n_steps=n_steps)
print('X', X.shape, 'y true', y_true.shape, 'y noise', y_noise.shape, 
      'intercepts', len(intercepts), intercepts[0].shape,
      'derivatives', len(derivatives), derivatives[0].shape, 
      'integrations', len(integrations), integrations[0].shape)

X (10000, 210) y true (10000, 1) y noise (10000, 1) intercepts 10 (10000,) derivatives 10 (10000,) integrations 10 (10000,)


In [17]:
intercepts = np.stack(intercepts, axis=1)
derivatives = np.stack(derivatives, axis=1)
integrations = np.stack(integrations, axis=1)

In [20]:
y = y_true + y_noise

In [21]:
X_train, X_test, \
y_train, y_test, \
y_true_train, y_true_test, \
intercepts_train, intercepts_test, \
derivatives_train, derivatives_test, \
integrations_train, integrations_test \
= train_test_split(X, y, y_true, intercepts, derivatives, integrations, test_size=test_size, random_state=seed)

# Iterative Method

In [24]:
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE

In [28]:
def get_estimator(method):
    if method =='linear':
        estimator = LinearRegression()
    elif method == 'SVM':
        estimator = SVR(kernel = 'linear') 
    elif method =='DT':
        estimator = DecisionTreeRegressor()
    else:
        raise 'wrong method'
    return estimator

In [36]:
import pickle
def calculate_precision_and_recall(a, b):
    set_a = set(a)
    set_b = set(b)
    intersection_count = len(set_a & set_b)
    recall = intersection_count / len(set_a)
    precision = intersection_count / len(set_b)
    
    return precision, recall

for formula_id in range(10,15):
    print(f'now executing formula: {formula_id}')
    num_features, function = formulas[formula_id]
    num_noises = 150
    num_data = 10000
    X_var = 0.33
    y_var = 0.01
    seed = 42
    test_size = 0.2
    n_steps = 20
    X, y_true, y_noise, intercepts, derivatives, integrations = create_simulation_data(function, num_features, num_noises, num_data, X_var, y_var, n_steps=n_steps)
    print('X', X.shape, 'y true', y_true.shape, 'y noise', y_noise.shape, 
          'intercepts', len(intercepts), intercepts[0].shape,
          'derivatives', len(derivatives), derivatives[0].shape, 
          'integrations', len(integrations), integrations[0].shape)
    gt_features = np.array([i for i in range(num_features)]) # useful features are the first num_features
    num_select = num_features # select exact this number of features
    repeats_times = 1
    intercepts = np.stack(intercepts, axis=1)
    derivatives = np.stack(derivatives, axis=1)
    integrations = np.stack(integrations, axis=1)
    y = y_true + y_noise
    X_train, X_test, \
    y_train, y_test, \
    y_true_train, y_true_test, \
    intercepts_train, intercepts_test, \
    derivatives_train, derivatives_test, \
    integrations_train, integrations_test \
    = train_test_split(X, y, y_true, intercepts, derivatives, integrations, test_size=test_size, random_state=seed)
    # for method in ['linear','DT']:
    for method in ['SVM']:
        for repeat in range(1,repeats_times+1):
            

            reduce_rate = 0.5
            num_cur_features = num_features+num_noises
            select_arr = np.ones(num_cur_features)
            print(np.sum(select_arr))
            remaining_inds = np.nonzero(select_arr)[0]
            resuming = True
            score_list = []
            precision_list =[]
            recall_list = []
            feature_list = []
            start_time = time.time()
            while resuming:
                bool_arr = np.array(select_arr, dtype='bool') 
                X_train_selected = X_train[...,bool_arr]
                X_test_selected = X_test[...,bool_arr]
                clf = get_estimator(method)
                clf.fit(X_train_selected, np.squeeze(y_train))

                y_pred = clf.predict(X_test_selected)
                score = mean_absolute_error(y_pred, y_test)
                selected_features = np.where(select_arr==1)[0]
    
                precision, recall = calculate_precision_and_recall(gt_features, selected_features)
                print(f'With {num_cur_features} features {method} mae score is {score},precision is {precision}, recall is {recall}')

                score_list.append(score)
                precision_list.append(precision)
                recall_list.append(recall)
                feature_list.append(selected_features)
                
                num_remove = int(num_cur_features*(1-reduce_rate))
                if num_cur_features - num_remove<=num_select:
                    num_remove = num_cur_features - num_select
                num_cur_features -= num_remove
                # print(num_cur_features)
                estimator = get_estimator(method)
                if num_remove == 0:
                    resuming = False
                    continue
                selector = RFE(estimator, n_features_to_select=num_cur_features,step=num_remove)
                selector = selector.fit(X_train_selected, np.squeeze(y_train))
                selected_RFE = selector.ranking_
                selected_RFE[selected_RFE != 1] = 0
                bool_arr = np.array(selected_RFE, dtype='bool') 
    
                inds = np.where(selected_RFE==0)[0]
                # print(inds)
                inds_to_remove = remaining_inds[inds]
                # print(inds_to_remove)
                select_arr[inds_to_remove] = 0
                remaining_inds = np.nonzero(select_arr)[0]

            elapsed_time = time.time() - start_time
            dict_method = {}
            dict_method['score'] = score_list
            dict_method['precision'] = precision_list
            dict_method['recall'] = recall_list
            dict_method['feat'] = feature_list
            dict_method['time'] = elapsed_time
            with open(f'./result_simulation/{formula_id}_{method}_{repeat}.pkl', 'wb') as f:
                pickle.dump(dict_method, f)

now executing formula: 10
X (10000, 156) y true (10000, 1) y noise (10000, 1) intercepts 6 (10000,) derivatives 6 (10000,) integrations 6 (10000,)
156.0
With 156 features SVM mae score is 0.0898943777370214,precision is 0.038461538461538464, recall is 1.0
With 78 features SVM mae score is 0.08964109956137514,precision is 0.07692307692307693, recall is 1.0
With 39 features SVM mae score is 0.08960915519046392,precision is 0.15384615384615385, recall is 1.0
With 20 features SVM mae score is 0.0896066093762312,precision is 0.25, recall is 0.8333333333333334
With 10 features SVM mae score is 0.08966455513540197,precision is 0.5, recall is 0.8333333333333334
With 6 features SVM mae score is 0.0896865667649549,precision is 0.8333333333333334, recall is 0.8333333333333334
now executing formula: 11
X (10000, 157) y true (10000, 1) y noise (10000, 1) intercepts 7 (10000,) derivatives 7 (10000,) integrations 7 (10000,)
157.0
With 157 features SVM mae score is 0.10236473372490366,precision is 0.0